In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as plt

In [2]:
df_c = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [21]:
df = df_c.copy()

In [22]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [9]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [24]:
df = df[~(df["StockCode"] == "POST")]

In [27]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [26]:
df.dropna(inplace = True)

C:\Users\90535\AppData\Local\Temp\ipykernel_10244\1791587065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


In [30]:
df = df[~(df["Invoice"].str.contains("C",na=False))]

In [31]:
df = df[df["Quantity"] > 0]

In [32]:
df = df[df["Price"] > 0]

In [33]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,13.016349,179.579125,1.000,2.00,6.00,12.00,80995.00
Price,396785.0,3.037677,17.829741,0.001,1.25,1.95,3.75,4161.06
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.00


In [34]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [35]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [36]:
replace_with_thresholds(df, "Quantity")

In [37]:
replace_with_thresholds(df, "Price")

In [38]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,11.855703,25.553760,1.000,2.00,6.00,12.00,298.50
Price,396785.0,2.835605,2.982371,0.001,1.25,1.95,3.75,31.56
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.00


In [39]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [40]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [41]:
def create_rules(dataframe, id=True, country="France"):
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules

In [44]:
from mlxtend.frequent_patterns import apriori, association_rules

In [48]:
rules = create_rules(df , country = "Germany")

C:\Users\90535\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [49]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(16237),(22326),0.011136,0.249443,0.011136,1.000000,4.008929,0.008358,inf
1,(22326),(16237),0.249443,0.011136,0.011136,0.044643,4.008929,0.008358,1.035073
2,(20674),(20675),0.022272,0.033408,0.013363,0.600000,17.960000,0.012619,2.416481
3,(20675),(20674),0.033408,0.022272,0.013363,0.400000,17.960000,0.012619,1.629547
4,(20674),(20676),0.022272,0.037862,0.011136,0.500000,13.205882,0.010293,1.924276
...,...,...,...,...,...,...,...,...,...
18365,(22629),"(22467, 22326, 22423, 21915, 22077)",0.104677,0.011136,0.011136,0.106383,9.553191,0.009970,1.106586
18366,(22326),"(22467, 22629, 22423, 21915, 22077)",0.249443,0.011136,0.011136,0.044643,4.008929,0.008358,1.035073
18367,(22423),"(22467, 22629, 22326, 21915, 22077)",0.140312,0.011136,0.011136,0.079365,7.126984,0.009573,1.074111
18368,(21915),"(22467, 22629, 22326, 22423, 22077)",0.046771,0.011136,0.011136,0.238095,21.380952,0.010615,1.297884


In [50]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [52]:
arl_recommender(rules, 22492, 1)

[21915, 22328, 22331, 22328, 22328, 22551, 22326, 22556, 22326]

In [53]:
check_id(df, 21915)

['RED  HARMONICA IN BOX ']
